In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Stomach_Cancer/GSE147163'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular subtypes in gastric cancer. [III]"
!Series_summary	"We identified the molecular subtypes and conserved modules in gastric cancer by unsupervised clustering algorithm. We defined six molecular signatrues of gastric cancer associated with the biological heterogeneity of gastric cancer and clinical outcome of patients."
!Series_overall_design	"We obtained gene expression profile of gastrectomy samples from 401 gastric cancer patients by HumanHT-12 v3.0 Expression BeadChip array (Illumina). Total RNA was extracted from the fresh-frozen gastrectomy specimens at the Yonsei University Severance Hospital (South Korea) between 2000 and 2010."
Sample Characteristics Dictionary:
{0: ['tissue: gastric cancer']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Given the description, the dataset contains gene expression data

# Find the appropriate rows for variables
trait_row = 0 # Given the sample characteristic 'tissue: gastric cancer'
age_row = None  # Not available in given data
gender_row = None  # Not available in given data

# Define the data conversion functions 

# Convert Stomach_Cancer trait
def convert_trait(value):
    if ':' in value:
        value = value.split(':')[1].strip()
    if value.lower() in ['cancer', 'gastric cancer']:
        return 1
    if value.lower() in ['normal', 'healthy']:
        return 0
    return None

# Convert age
def convert_age(value):
    if ':' in value:
        value = value.split(':')[1].strip()
    try:
        return float(value)
    except ValueError:
        return None

# Convert gender
def convert_gender(value):
    if ':' in value:
        value = value.split(':')[1].strip().lower()
    if value in ['male', 'm']:
        return 1
    if value in ['female', 'f']:
        return 0
    return None

# Saving cohort information
save_cohort_info('GSE147163', './preprocessed/Stomach_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Stomach_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Stomach_Cancer/trait_data/GSE147163.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4419484': [1], 'GSM4419485': [1], 'GSM4419486': [1], 'GSM4419487': [1], 'GSM4419488': [1], 'GSM4419489': [1], 'GSM4419490': [1], 'GSM4419491': [1], 'GSM4419492': [1], 'GSM4419493': [1], 'GSM4419494': [1], 'GSM4419495': [1], 'GSM4419496': [1], 'GSM4419497': [1], 'GSM4419498': [1], 'GSM4419499': [1], 'GSM4419500': [1], 'GSM4419501': [1], 'GSM4419502': [1], 'GSM4419503': [1], 'GSM4419504': [1], 'GSM4419505': [1], 'GSM4419506': [1], 'GSM4419507': [1], 'GSM4419508': [1], 'GSM4419509': [1], 'GSM4419510': [1], 'GSM4419511': [1], 'GSM4419512': [1], 'GSM4419513': [1], 'GSM4419514': [1], 'GSM4419515': [1], 'GSM4419516': [1], 'GSM4419517': [1], 'GSM4419518': [1], 'GSM4419519': [1], 'GSM4419520': [1], 'GSM4419521': [1], 'GSM4419522': [1], 'GSM4419523': [1], 'GSM4419524': [1], 'GSM4419525': [1], 'GSM4419526': [1], 'GSM4419527': [1], 'GSM4419528': [1], 'GSM4419529': [1], 'GSM4419530': [1], 'GSM4419531': [1], 'GSM4419532': [1], 'GSM4419533': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052',
       'ILMN_1343059', 'ILMN_1343061', 'ILMN_1343062', 'ILMN_1343063',
       'ILMN_1343064', 'ILMN_1343291', 'ILMN_1343295', 'ILMN_1343296',
       'ILMN_1343297', 'ILMN_1343298', 'ILMN_1343299', 'ILMN_1343301',
       'ILMN_1343302', 'ILMN_1343303', 'ILMN_1343304', 'ILMN_1343305'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B', 'TRIM44', 'LOC653895'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'RefSeq_ID': ['XM_933824.1', nan, 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'Unigene_ID': [nan, 'Hs.575038', nan, nan, nan], 'Entrez_Gene_ID': [23117.0, nan, 2213.0, 54765.0, 653895.0], 'GI': [89040007.0, 10437021.0, 88952550.0, 29029528.0, 89033487.0], 'Access

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Extract the keys for identifier and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and generate the resulting gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Stomach_Cancer/gene_data/GSE147163.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Stomach_Cancer')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE147163', './preprocessed/Stomach_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Stomach_Cancer/GSE147163.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Stomach_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Stomach_Cancer' in this dataset is severely biased.

